In [1]:
import pandas as pd
import pickle as pkl
import numpy as np

from tqdm.notebook import tqdm

In [2]:
sample_ids = list(pd.read_csv('data/seen_objects_all_source_id.csv').sample(n=1000000, random_state=42)['source_id'].values)

In [3]:
for i in tqdm(range(100)):
    with open(f'data/listings_{i}.pkl', 'rb') as f:
        tmp_id = pkl.load(f)['rebny_id']
    for j in tmp_id:
        sample_ids.remove(j)
sample_ids

  0%|          | 0/100 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'data/listings_0.pkl'

In [ ]:
len(sample_ids)

209

In [11]:
#from numpy import float64
cols = []
float_cols = []
val_errors = []
type_errors = set()
t = type(pd.to_numeric('1.0'))
for i in tqdm(range(100)):
    with open(f'data/seen_objects_{i}.pkl', 'rb') as f:
        tmp_df = pkl.load(f)

    for d in tmp_df['data']:
        cols += list(d.keys())
        for k in d.keys():
            try:
                #if k in ['Latitude', 'Longitude']:
                 #   print(f'{k}: {d[k]} {type(pd.to_numeric(d[k]))}')
                if type(pd.to_numeric(d[k])) == np.float64 and (not pd.isna(d[k])):
                    #if k in ['Latitude', 'Longitude']:
                     #   print(type(pd.to_numeric(d[k])))
                    float_cols.append(k)
            except:
                continue
    float_cols = list(set(float_cols))
    cols = list(set(cols))
    val_errors = list(set(val_errors))

# cols = set(cols)
len(cols)    

  0%|          | 0/100 [00:00<?, ?it/s]

714

In [82]:
type_errors

{'Media',
 '_PropertyADU',
 '_PropertyAdu',
 '_PropertyGreenVerification',
 '_PropertyPowerProduction',
 '_PropertyUnits'}

In [107]:
type(np.nan)

float

In [83]:
val_errors

['ListOfficeFax',
 'ElevationUnits',
 'BelowGradeFinishedAreaUnits',
 'DistanceToShoppingComments',
 'OriginalEntryTimestamp',
 'EstimatedCoBuyerOfficeAOR',
 'ListOfficePhone',
 'OwnerPhone',
 'BusinessType',
 'StateOrProvince',
 'SecurityFeatures',
 'PossibleUseComments',
 'ListOfficeMlsId',
 'ListingId',
 'AssociationPhoneExt',
 'Cleared',
 'AvailabilityDate',
 'TaxOtherAnnualAssessmentAmountSource',
 'CloseDate',
 'DistanceToFreewayComments',
 'LockBoxVersion',
 'LockBoxSerialNumber',
 'CommonWalls',
 'ListTeamEmail',
 'BuyerAgencyCompensationPer',
 'CoBuyerOfficeName',
 'PriceChangeTimestamp',
 'CoListAgentLastName',
 'CoBuyerOfficeStateLicense',
 'EstimatedBuyerOfficeStateLicense',
 'Trees',
 'CoListAgentTollFreePhone',
 'City',
 'EstimatedBuyerTeamName',
 'URLDescription',
 'AssociationName',
 'EstimatedBuyerAgentAOR',
 'CommunityFeatures',
 'PossibleUse',
 'ListAgentStateLicense',
 'Occupancy',
 'ElementarySchool',
 'Average2BedAreaUnits',
 'HighSchoolDistrict',
 'ProbateAuthori

In [109]:
with open('seen_objects_float_cols.txt', 'w') as f:
    print(float_cols, file=f)
len(float_cols)

187

In [71]:
raw_df = pd.DataFrame({'in_seen_objects': cols})

In [7]:
raw_df.head()

,in_seen_objects
0,CoListAgentFirstName
1,BusinessURL
2,NumberWithDrapes
3,BelowGradeFinishedArea
4,ListAgentMobilePhone


In [87]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pickle as pkl

import time

def pull_data(sql_query:str, file_out=None, port_num=2023, db_name='public'):
    """
    For a given sql_query, this function will connect to Perchwell's database, pull
    the corresponding data, save it to a csv in the file specified (default is 
    pw.csv), and returns that data in the form of a Pandas DataFrame.

    Args:
        sql_query (str): query whose results you wish to download
        file_out (str, optional): filepath for the output file. If None, then
            no file is created. Defaults to None.
        port_num (int, optional): Port number for Teleport Connection. Defaults
            to 2023.

    Returns:
        pd.DataFrame: DataFrame containing results of sql_query 
    """    
    if sql_query[-1] == ';':
        sql_query = sql_query[:-1]
    pw_df = []
    current_row = 0
    # Create the SQLAlchemy engine
    engine = create_engine(\
        f"postgresql://teleport:@localhost:{port_num}/{db_name}")

    query_string = f"""{sql_query}
        -- NOTE: DO NOT DELETE BELOW THIS
        LIMIT 10000 OFFSET {current_row};
        """
    start = time.time()
    try:
        # Execute the query
        print(f'new offset: {current_row}')
        with engine.connect() as conn:
            result = conn.execute(text(query_string))

            # Process the query result
            for row in result:
                # Access the row data
                pw_df.append(row)
    except:
        # in case of timeout
        engine = create_engine(\
            f"postgresql://teleport:@localhost:{port_num}/{db_name}")
        with engine.connect() as conn:
            result = conn.execute(text(query_string))
            for row in result:
                pw_df.append(row)
    pw_df = pd.DataFrame(pw_df)
    if file_out:
        pw_df.to_csv(file_out)
    print(f'time taken: {time.time() - start}')
    return pw_df

In [10]:
with open(f'data/reso_0.pkl', 'rb') as f:
    cols = list(set(pkl.load(f).columns))
reso_df = pd.DataFrame({'in_reso': cols})

FileNotFoundError: [Errno 2] No such file or directory: 'data/reso_0.pkl'

In [91]:
with open(f'gen_queries/reso/reso_data_0.sql', 'r') as f:
    sql = f.read()
reso_df = pull_data(sql, 'data/reso_0.csv', db_name='perchwell')
reso_df.dtypes

new offset: 0
time taken: 14.937081098556519


listing_id                        object
accessibility_features            object
additional_parcels_description    object
additional_parcels_y_n            object
anchors_co_tenants                object
                                   ...  
standard_status                   object
state_or_province                 object
street_dir_prefix                 object
street_dir_suffix                 object
year_built_source                 object
Length: 357, dtype: object

In [9]:
num_columns = []
for col in reso_df.columns:
    if reso_df[col].dtype == np.float64:
        num_columns.append(col)

len(set(num_columns))
with open('float_cols_reso.txt', 'w') as f:
    print(set(num_columns), file=f)

NameError: name 'reso_df' is not defined

In [103]:
with open('gen_queries/listings/listings_data_0.sql', 'r') as f:
    sql = f.read()
listings_df = pull_data(sql, db_name='perchwell')
num_columns = []
for col in listings_df.columns:
    if listings_df[col].dtype == np.float64:
        num_columns.append(col)
len(set(num_columns))
with open('float_cols_babylon.txt', 'w') as f:
    print(set(num_columns), file=f)

new offset: 0
time taken: 2.8987600803375244


In [104]:
with open('gen_queries/features/features_data_0.sql', 'r') as f:
    sql = f.read()
features_df = pull_data(sql, db_name='perchwell')
num_columns = []
for col in features_df.columns:
    if features_df[col].dtype == np.float64:
        num_columns.append(col)

len(set(num_columns))
with open('float_cols_babylon.txt', 'a') as f:
    print(set(num_columns), file=f)

new offset: 0
time taken: 35.16295599937439


In [9]:
with open(f'data/listings_0.pkl', 'rb') as f:
    cols = list(pkl.load(f).columns)
with open('data/features_0.pkl', 'rb') as f:
    cols += list(pkl.load(f).columns)
babylon_df = pd.DataFrame({'in_babylon': list(set(cols))})


In [8]:
import re
def column_converter(df, col:str='', out_col:str='', conversion_type:str='lower'):
     assert (col !='') or (out_col !='')
     assert (conversion_type =='lower') or (conversion_type =='upper')
     out_lst = []
     if conversion_type=='lower':
          vals = df[col].to_list()
          for col in vals:
            tmp_val = "_".join(re.split('(?<=.)(?=[A-Z])', col)).lower()
            out_lst.append(tmp_val)
          df[out_col] = out_lst
          return df
     elif conversion_type=='upper':
         df[out_col] = df[col].str.split("_")
         vals = df[out_col].to_list()
         for col in vals:
            tmp_val = [x.capitalize() for x in col]
            out_lst.append(''.join(tmp_val))
         df[out_col] = out_lst
         return df
     else:
         print('select lower or upper for conversion_type') 

In [17]:
column_converter(pd.DataFrame({'float_cols': float_cols}), col='float_cols', out_col='snake_case').to_csv('float_cols_seen_objects.csv', index=False)

In [11]:
raw_df = column_converter(raw_df,'in_seen_objects', 'transformed_col')
raw_df.head()

,in_seen_objects,transformed_col
0,CoListAgentFirstName,co_list_agent_first_name
1,BusinessURL,business_u_r_l
2,NumberWithDrapes,number_with_drapes
3,BelowGradeFinishedArea,below_grade_finished_area
4,ListAgentMobilePhone,list_agent_mobile_phone


In [12]:
col_comparison = raw_df.merge(reso_df, how='left', left_on='transformed_col', right_on='in_reso')
col_comparison[pd.isna(col_comparison['in_reso'])]

,in_seen_objects,transformed_col,in_reso
1,BusinessURL,business_u_r_l,NaN
2,NumberWithDrapes,number_with_drapes,NaN
5,ParkManagerName,park_manager_name,NaN
6,BuyerOfficeStateLicense,buyer_office_state_license,NaN
7,LaundryIncome,laundry_income,NaN
...,...,...,...
703,LandLeaseTransferFee,land_lease_transfer_fee,NaN
704,LongitudeManual,longitude_manual,NaN
706,LeaseRenewalOptionYN,lease_renewal_option_y_n,NaN
708,AssociationPhone3Ext,association_phone3_ext,NaN


In [13]:
reso_comparison = reso_df.merge(raw_df, how='left', right_on='transformed_col', left_on='in_reso')
reso_comparison[pd.isna(reso_comparison['in_seen_objects'])]

,in_reso,in_seen_objects,transformed_col


In [14]:
col_comparison[pd.isna(col_comparison['in_reso'])].to_csv('in_seen_objects_not_reso.csv', index=False)

In [15]:
pw_check = babylon_df.merge(reso_df, how='left', right_on='in_reso',left_on='in_babylon')

In [16]:
reso_check = reso_df.merge(babylon_df, how='left', left_on='in_reso',right_on='in_babylon')
reso_check[pd.isna(reso_check['in_babylon'])]

,in_reso,in_babylon
0,room_type,NaN
2,distance_to_bus_comments,NaN
3,fireplace_y_n,NaN
7,list_office_key_numeric,NaN
8,virtual_tour_u_r_l_branded,NaN
...,...,...
344,contract_status_change_date,NaN
345,documents_change_timestamp,NaN
347,license3,NaN
350,distance_to_electric_comments,NaN


In [17]:
pw_check[pd.isna(pw_check['in_reso'])]

,in_babylon,in_reso
18,description,NaN
29,rebny_id,NaN
143,security_expense,NaN


Only in babylon:
    - description
    - security_expense
    - rebny_id

## Checking other objects in seen_objects

In [19]:
adu = pd.read_csv('data/audit_2/ADU_sample_2023_12_18.csv')

In [20]:
adu.head()

,source_id,data
0,64431,"{'DeletedYN': '0', 'OriginatingSystemID': 'SAN..."
1,59314,"{'DeletedYN': '0', 'OriginatingSystemID': 'SAN..."
2,78518,"{'DeletedYN': '0', 'OriginatingSystemID': 'SAN..."
3,71963,"{'DeletedYN': '0', 'OriginatingSystemID': 'SAN..."
4,78610,"{'DeletedYN': '0', 'OriginatingSystemID': 'SAN..."


In [34]:
import json
def get_col_counts(df):
    col_counts = {}
    errors = 0
    for d in df['data']:
        try:
            for key in json.loads(d.replace("'",'"')).keys():
                try:
                    col_counts[key] += 1
                except:
                    col_counts[key] = 1
        except json.JSONDecodeError:
            errors += 1

    print(f'num in sample: {len(df) - errors}')

    return col_counts, len(df) - errors

In [2]:
def get_raw_col_counts(df):
    col_counts = {}
    errors = 0
    for d in df['data']:
        try:
            for key in d.keys():
                try:
                    col_counts[key] += 1
                except:
                    col_counts[key] = 1
        except:
            errors += 1

    #print(f'num in sample: {len(df) - errors}')

    return col_counts, len(df) - errors

In [42]:
import os
output = []
n = []
for f in os.listdir('data/audit_2/'):
    print(f)
    i, j = get_col_counts(pd.read_csv(f'data/audit_2/{f}'))
    output.append(i)
    n.append(j)

HISTORY_sample_2023_12_18.csv
num in sample: 500000
UNIT_sample_2023_12_18.csv
num in sample: 499996
ADU_sample_2023_12_18.csv
num in sample: 500000
OFFICE_sample_2023_12_18.csv
num in sample: 499483
GREEN_sample_2023_12_18.csv
num in sample: 500000
POWER_sample_2023_12_18.csv
num in sample: 500000
AGENT_sample_2023_12_18.csv
num in sample: 499778
OPEN_HOUSE_sample_2023_12_18.csv
num in sample: 498018


In [3]:
output = []
n = []
for num in tqdm(range(100)):
    with open(f'data/seen_objects_{num}.pkl', 'rb') as f:
        i, j = get_raw_col_counts(pkl.load(f))
    output.append(i)
    n.append(j)

  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
output[-1], n[-1]

({'City': 9928,
  'Roof': 5506,
  'Zoning': 4353,
  'Cooling': 8031,
  'Latitude': 9016,
  'CloseDate': 6224,
  'CoolingYN': 8402,
  'DeletedYN': 10000,
  'LandValue': 153,
  'ListPrice': 9989,
  'ListingId': 10000,
  'Longitude': 9017,
  'ParkingYN': 6426,
  'YearBuilt': 9158,
  'Appliances': 5851,
  'AutoSoldYN': 9981,
  'ClosePrice': 6278,
  'Directions': 6868,
  'EndingDate': 8737,
  'PostalCode': 9881,
  'StreetName': 9993,
  'TenantPays': 143,
  'GrossEquity': 139,
  'LotSizeArea': 5800,
  'PhotosCount': 10000,
  'AppliancesYN': 6749,
  'CurrentPrice': 9986,
  'DaysOnMarket': 9728,
  'ListAgentAOR': 9742,
  'ListAgentFax': 5088,
  'ListingTerms': 7007,
  'LotSizeAcres': 5711,
  'LotSizeUnits': 8517,
  'MLSAreaMajor': 9410,
  'ParcelNumber': 9427,
  'ParkingTotal': 7573,
  'PoolFeatures': 7276,
  'PropertyType': 10000,
  'StoriesTotal': 4074,
  'StreetSuffix': 6761,
  'BuyerAgentAOR': 5318,
  'ContactOrder1': 7836,
  'ContactOrder2': 6954,
  'ContactOrder3': 5629,
  'ContactOrder4

In [5]:
count = 0
for o in output:
    for k in o.keys():
        o[k] = [o[k], n[count]]
    count += 1


In [7]:
for k in output[-1].keys():
    if 'tax' in k.lower():
        print(k, output[-1][k])

NewTaxesExpense [176, 10000]
TaxLot [5565, 10000]
TaxOtherAnnualAssessmentAmount [691, 10000]
TaxModel [564, 10000]
TaxRate [166, 10000]
TaxYear [162, 10000]
TaxRateTotal [102, 10000]
TaxAnnualAmount [130, 10000]
TaxTract [1154, 10000]
TaxBlock [331, 10000]
TaxTractNumber [4745, 10000]
TaxOtherAnnualAssessmentAmountSource [299, 10000]
TaxesPercent [54, 10000]
TaxCensusTract [1215, 10000]
TaxArea [71, 10000]
RealEstateTaxExpense [3, 10000]
TaxMapNumber [1, 10000]


In [47]:
retval = {}
names = os.listdir('data/audit_2/')
names = [x.split('_')[0] for x in names]
for i in range(len(output)):
    tmp = pd.DataFrame(output[i]).T
    tmp.columns = ['num_not_null', 'num_in_sample']
    tmp['pct_data'] = tmp['num_not_null'] / tmp['num_in_sample']
    retval[names[i]] = tmp

retval

{'HISTORY':                            num_not_null  num_in_sample  pct_data
 NewValue                          50000          50000   1.00000
 ClassName                         50000          50000   1.00000
 FieldName                         50000          50000   1.00000
 ResourceName                      50000          50000   1.00000
 PreviousValue                     27573          50000   0.55146
 TransactionType                   50000          50000   1.00000
 HistoryKeyNumeric                 50000          50000   1.00000
 OriginatingSystemID               50000          50000   1.00000
 ModificationTimestamp             50000          50000   1.00000
 ResourceRecordKeyNumeric          50000          50000   1.00000
 ChangedByMemberKeyNumeric         20706          50000   0.41412,
 'UNIT':                           num_not_null  num_in_sample  pct_data
 DeletedYN                        49996          49996  1.000000
 UnitTypeType                     49996          49996  1.

In [51]:
def report_writer(dfs:dict, report_name:str):
    """Creates an excel report using a dictionary of dfs.

    Args:
        dfs (dict): Data
        report_name (str): Name of file.
    """
    writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
    workbook = writer.book
    format = workbook.add_format()
    format.set_align('center')
    for sheetname, df in dfs.items():
        df.to_excel(writer, sheet_name=sheetname)
        worksheet = writer.sheets[sheetname]
        for idx, col in enumerate(df):
            series =df[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
            ))+1
            worksheet.set_column(idx, idx, max_len, format)

    writer.close()

In [52]:
report_writer(retval, '50k_sample_seen_objects.xlsx')

In [53]:
reso_check[pd.isna(reso_check['in_babylon'])].to_clipboard(index=False)

In [54]:
reso_check.to_clipboard(index=False)

In [55]:
pw_check.to_clipboard(index=False)

In [56]:
mapping = pd.read_csv('mapping_3rd_edit.csv')

In [57]:
mapping.head()

,CRMLS_field,CRMLS_lookup,PW_field,PW_lookup,localizedField
0,OriginatingSystemID,NaN,originating_system_id,NaN,True
1,CountyOrParish,NaN,CountyOrParish,NaN,False
2,AssociationFee2,NaN,AssociationFee2,NaN,False
3,BuyerAgentAOR,NaN,BuyerAgentAOR,NaN,False
4,BuyerOfficeAOR,NaN,BuyerOfficeAOR,NaN,False


In [61]:
raw_mapping = raw_df.merge(mapping, how='left', left_on='in_seen_objects',right_on='CRMLS_field')
raw_mapping[pd.isna(raw_mapping['CRMLS_field'])]['in_seen_objects'].to_clipboard(index=False)

In [87]:
not_in_mapping = list(raw_mapping[pd.isna(raw_mapping['CRMLS_field'])]['in_seen_objects'])
print(len(not_in_mapping))
#not_in_mapping.remove('Media')


153


In [88]:
final = {}
for k in not_in_mapping:
    final[k] = [0, 0]
for i in tqdm(range(100)):
    with open(f'data/seen_objects_{i}.pkl', 'rb') as f:
        tmp_df = pkl.load(f)
    for d in tmp_df['data']:
        for k in final.keys():
            try:
                if pd.isna(d[k]):
                    final[k][0] += 1
                    final[k][1] += 1
                else:
                    final[k][1] += 1
            except KeyError:
                final[k][0] += 1
                final[k][1] += 1
            except ValueError:
                if pd.isna(d[k]).all():
                    final[k][0] += 1
                    final[k][1] += 1
                else:
                    final[k][1] += 1
final = pd.DataFrame(final).T
final.columns = ['null', 'total']
final.head()

  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_69271/760699597.py:10: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(d[k]):
/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_69271/760699597.py:10: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(d[k]):
/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_69271/760699597.py:10: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(d[k]):
/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_69271/760699597.py:10: DeprecationWarning: The truth value of an empty arr

,null,total
BuyerOfficeStateLicense,803336,1000000
InternetAddressDisplayYN,42598,1000000
CoListTeamEmail,999935,1000000
PricePerLotSquareFoot,962456,1000000
Neighborhood,851832,1000000


In [89]:
final.to_clipboard()